In [1]:
import pandas as pd
import numpy as np

Read the CSVs

In [2]:
order = pd.read_csv('order.csv')
purchase = pd.read_csv('purchase.csv')

purchase.shape

(6686, 16)

## Construct the Vlookup column
The lookup column is by combining the fc_id_dv_id_distributor_id

In [3]:
order['lookup_index'] = order['fc_id'].astype(str) + '_' + order['drug_variation_id'].astype(str) + '_' + order['distributor_name'].astype(str)

purchase['lookup_index'] = purchase['fc_id'].astype(str) + '_' + purchase['drug_variation_id'].astype(str) + '_' + purchase['distributor_name'].astype(str)

Drop duplicates based on the lookup_index of order data

In [4]:
order = order.drop_duplicates(subset=['lookup_index'])

In purchase, pull in the order loose quantity via the lookup column

In [5]:
purchase = purchase.merge(order[['loose_quantity', 'lookup_index']], how = 'left', on = 'lookup_index', suffixes=(None, '_ordered'))
purchase.shape

(6686, 18)

Replace NaN with 0 and set the analysis column value to 'Not ordered'
We do this first by duplicating the loose_quantity_y column to a new column called analysis and then updating the NaN values

In [6]:
purchase['analysis'] = purchase['loose_quantity_ordered']
purchase['loose_quantity_ordered'] = purchase['loose_quantity_ordered'].fillna(0)
purchase['analysis'] = purchase['analysis'].fillna('Not ordered')

Set the excess, shortage and exact values for the rest of the analysis columns

In [7]:
purchase['analysis'].loc[(purchase['analysis'] != 'Not ordered') & (purchase['loose_quantity'] > purchase['loose_quantity_ordered'])] = 'Excess'

In [8]:
purchase['analysis'].loc[(purchase['analysis'] != 'Not ordered') & (purchase['loose_quantity'] < purchase['loose_quantity_ordered'])] = 'Shortage'

In [9]:
purchase['analysis'].loc[(purchase['analysis'] != 'Not ordered') & (purchase['loose_quantity'] == purchase['loose_quantity_ordered'])] = 'Equal'

Output the file

In [10]:
purchase.to_csv('output.csv')

In [11]:
table = pd.pivot_table(purchase, index=['distributor_name'], columns=['analysis'], values="loose_quantity", aggfunc='count')
table

analysis,Equal,Excess,Not ordered,Shortage
distributor_name,,,,
Bhagerathi Enterprises,17.0,12.0,NaN,1.0
M Ratan Agencies,37.0,20.0,32.0,2.0
MAHAVEER MEDI-SALES PVT. LTD.,2755.0,222.0,843.0,117.0
MEHUL AGENCIES,434.0,206.0,55.0,57.0
Nikhil Enterprises,13.0,2.0,2.0,NaN
PADMAVATI ENTERPRISES,27.0,NaN,20.0,1.0
Shubh Enterprises,40.0,7.0,1.0,NaN
VARDHAMAN PHARMA Distributors Private Limited,1177.0,78.0,452.0,56.0


In [12]:
table.to_csv('pivot.csv',index_label=[table.index.name])